### API Key
Pc9BW2uXp3M0ctS3W7g5vlG9le7BmRxslFxK7kcK

In [1]:
import requests
import json
import hashlib

### api call outline

### api route builder

In [2]:
def build_metadata_request_url(dataset_id, api_key):
    base_url = "https://api.eia.gov/v2/"
    endpoint = f"{dataset_id}?api_key={api_key}"
    url = base_url + endpoint
    return url

In [11]:
def get_metadata(url):
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None


In [20]:
def build_data_request_url(dataset_id):
    base_url = "https://api.eia.gov/v2/"
    endpoint = f"{dataset_id}/data/?"
    url = base_url + endpoint
    return url

In [22]:
url = build_data_request_url("petroleum/pri/spt")

In [5]:
def get_child_metadata(api_key, dataset_id, child_dataset_id):
    base_url = "https://api.eia.gov/v2/"
    endpoint = f"{dataset_id}/{child_dataset_id}?api_key={api_key}"
    url = base_url + endpoint
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

In [6]:
get_child_metadata("Pc9BW2uXp3M0ctS3W7g5vlG9le7BmRxslFxK7kcK", "petroleum", 'pri')

{'response': {'id': 'pri',
  'name': 'Petroleum, Prices',
  'description': 'Petroleum, Prices',
  'routes': [{'id': 'gnd',
    'name': 'Weekly Retail Gasoline and Diesel Prices',
    'description': "Conventional area is any area that does not require the sale of reformulated gasoline. All types of finished motor gasoline may be sold in this area. RFG area is an ozone nonattainment area designated by the U.S. Environmental Protection Agency (EPA), which requires the use of reformulated gasoline. We stopped publishing Low Sulfur On-Highway Diesel (LSD) prices at the U.S. level on December 8, 2008, because fewer stations were selling LSD as a result of EPA diesel fuel regulations. We continued to collect LSD prices from retail outlets and included them in the Diesel Average All Types price until July 26, 2010, when no more outlets reported LSD sales. Beginning July 26, 2010, publication of Ultra Low Sulfur Diesel (ULSD) price became fully represented by the Diesel Average All Types price.

In [23]:
def fetch_generic_data(url, query_params):
    response = requests.get(url, params=query_params)
    print(response.url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("API request failed with status code:", response.status_code)
        return None

In [24]:
params = {
        "api_key": "Pc9BW2uXp3M0ctS3W7g5vlG9le7BmRxslFxK7kcK",
        "data[]": "value",
        "facets": {},
        "frequency": "weekly",
        "start": "2023-07-28",
        "end": "2023-07-28",
        "sort[0][column]": "period",
        "sort[0][direction]": "asc",
        "offset": 0,
        "length": 1
    }

fetch_generic_data(url, params)

https://api.eia.gov/v2/petroleum/pri/spt/data/?api_key=Pc9BW2uXp3M0ctS3W7g5vlG9le7BmRxslFxK7kcK&data%5B%5D=value&frequency=weekly&start=2023-07-28&end=2023-07-28&sort%5B0%5D%5Bcolumn%5D=period&sort%5B0%5D%5Bdirection%5D=asc&offset=0&length=1


{'response': {'warnings': [{'warning': 'incomplete return',
    'description': 'The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.'}],
  'total': 11,
  'dateFormat': 'YYYY-MM-DD',
  'frequency': 'weekly',
  'data': [{'period': '2023-07-28',
    'duoarea': 'Y05LA',
    'area-name': 'LOS ANGELES',
    'product': 'EPD2DC',
    'product-name': 'Carb Diesel',
    'process': 'PF4',
    'process-name': 'Spot Price FOB',
    'series': 'EER_EPD2DC_PF4_Y05LA_DPG',
    'series-description': 'Los Angeles, CA Ultra-Low Sulfur CARB Diesel Spot Price (Dollars per Gallon)',
    'value': 3.067,
    'units': '$/GAL'}]},
 'request': {'command': '/v2/petroleum/pri/spt/data/',
  'params': {'api_key': 'Pc9BW2uXp3M0ctS3W7g5vlG9le7BmRxslFxK7kcK',
   'data': ['value'],
   'frequency': 'weekly',
   'start': '2023-07-28',
   'end': '2023-07-28',
   'sort': [{'column': 'period', 'direction': 'asc'}],
   'offs

In [10]:
url = build_data_request_url("petroleum/pri/spt")
print(url)

https://api.eia.gov/v2/petroleum/pri/spt/data/?


In [25]:
def get_full_output(api_url):
    output = []
    offset = 0
    chunk_size = 150 # Initial chunk size

    while True:
        params = {
        "api_key": "Pc9BW2uXp3M0ctS3W7g5vlG9le7BmRxslFxK7kcK",
        "data[]": "value",
        "facets": {},
        "frequency": "weekly",
        "start": "1986-01-03",
        "end": "2023-07-28",
        "sort[0][column]": "period",
        "sort[0][direction]": "asc",
        "offset": offset,
        "length": chunk_size
    }
        response = requests.get(api_url, params=params)
        
        if response.status_code == 200:
            chunk = response.json()['response']['data']
            output += chunk
            offset += len(chunk)
            
            if len(chunk) < chunk_size:
                # If the returned chunk is smaller than the requested chunk size, we've reached the end
                break
        else:
            print(f"Error: Request failed with status code {response.status_code}")
            break

    return output

def save_to_file(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)
        
filename = hashlib.sha256(url.encode()).hexdigest() + ".json"  # Generate a filename based on URL hash
full_output_json = get_full_output(url)
save_to_file(filename, full_output_json)
print(f"Data saved to {filename}")


Data saved to ce389197fe870bdd91095d12fecd9d167197ad4d9258fd962eb82c5590abf22f.json
